# Import packages and libraries

In [ ]:
# Import to navigate and gather files 
import pathlib 
from pathlib import Path
import os

In [ ]:
# Import for FASTQ file parsing 
import Bio
from Bio import SeqIO

In [ ]:
# Import for data processing
import numpy as np
import pandas as pd
import statistics
from statistics import mean
import random
import math
from scipy.stats import norm
from scipy.stats import sem
from scipy.stats import linregress

In [ ]:
# Import for graphing
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from matplotlib import rc
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

# Obtain the list of FASTQ files to analyze

Each next-generation sequencing library encompassed one 96-well plate of samples. Some sequencing libraries were run on multiple lanes. Barcode recovery was not substantially affected by the lane on which a sequencing library was run. The following information explains the file naming conventions used for the SRA entries: 

Sequencing Run: 6512S - Moderna-vaccinated cohort samples + pre-pandemic & convalescent donor samples
- Lane 1: Plate 2 (Samples D23-17602 through D23-17697), Replicate 1 

Sequencing Run: 6540S - Moderna-vaccinated cohort samples + pre-pandemic & convalescent donor samples
- Lane 2: Plate 2 (Samples D23-17602 through D23-17697), Replicate 2
- Lane 3: Plate 1 (Samples D23-17505 through D23-17600), Replicate 1
- Lane 4: Plate 1 (Samples D23-17505 through D23-17600), Replicate 2

Sequencing Run: 6658S - Moderna-vaccinated cohort samples + pre-pandemic & convalescent donor samples
- Lane 1: Plate 3 (Samples D24-4494 through D24-4589), Replicate 1
- Lane 2: Plate 3 (Samples D24-4494 through D24-4589), Replicate 2
- Lane 3: Plate 4 (Samples D24-4591 through D24-4686), Replicate 1
- Lane 4: Plate 4 (Samples D24-4591 through D24-4686), Replicate 1

Sequencing Run: 6697S - J&J-vaccinated cohort samples
- Lane 1: Plate 5 (Samples D24-6269 through D24-6364)
- Lane 2: Plate 6 (Samples D24-6366 through D24-6461)
- Lane 3: Plate 7 (Samples D24-6463 through D24-6558)

Sequencing Run: 7182S - Neutralization assay samples
- All four lanes ran the same set of samples
- Only the 48/96 samples from this sequencing library that were from the neutralization assay are available on the SRA

In [ ]:
# Identify the current working directory
os.getcwd()

In [ ]:
# Specify the path for the folder containing the sequencing files 
# relative to the path of the current working directory 
seq_file_folder = '/folder_path'

In [ ]:
# Obtain a list of all the FASTQ files needed (separated by Read 1 and Read 2)

# Obtain Read 1 for each sample
# fastq_paths_r1 = sorted(Path(seq_file_folder).glob('**/*_1_sequence.fastq'))

# Obtain Read 2 for each sample 
fastq_paths_r2 = sorted(Path(seq_file_folder).glob('**/*_2_sequence.fastq'))


# Note: only Read 2 was used in our analyses. In Read 2, the barcode is located 
#       closer to the start of the read, placing it in a region where sequencing 
#       accuracy is likely to be higher. 

# Note: ** means all folders within the current folder. 
# Note: * means files ending in the text following * will be obtained.  

In [ ]:
# Check the number of file paths obtained 
print(f'Number of file paths: {len(fastq_paths_r2)}')

# Load information about the library barcodes and samples

In [ ]:
# Load the list of library member barcode assignments into a dataframe 
bc_df = pd.read_excel('Barcode List.xlsx')

# Note: the columns include Library Member Name, Barcode Number, 
# Barcode Sequence (Forward), Barcode Sequence (Reverse), and Type of Virus.

In [ ]:
# Load the list(s) of samples into a dataframe  
samp_df = pd.read_excel('Samples List.xlsx')

# Note: the sample names in the provided spreadsheet correspond to the end of each "Library Name" 
#       in the SRA. These sample names/"Library Name" ends are shared across the multiple lanes 
#       running the same plate/sequencing library. 

# Obtain barcode counts from the reads in the FASTQ files

Since some library member barcodes are separated by short Hamming distances, only read sequences that match barcodes exactly are counted in the final barcode counts for each library member.

For simplicity, this code assumes that all FASTQ files and reads are formatted correctly. A try-catch exception should be included if files and reads are likely to have errors.

In [ ]:
# Define a function that creates empty dataframes to store barcode counts and barcode recovery information 

# FUNCTION PARAMETERS
# lib_bc_list: list of the library member barcode sequences
# list_of_samps: list of the samples in the sequencing run 

def make_bc_dfs(list_of_samps, lib_bc_list): 
    
    # Create a dataframe to store the barcode counts 
    bc_counts = pd.DataFrame(index = lib_bc_list)
    
    # Create a dataframe to store information on barcode recovery  
    bc_recovery = pd.DataFrame(0, 
                               index = list_of_samps, 
                               columns = ['Number of Reads per Sample', 
                                          'Number of Barcodes Recovered per Sample',
                                          'Percent Barcode Recovery'])
    
    # Create a dataframe to store quality score information on reads 
    rd_qual_accepted = pd.DataFrame(index = list_of_samps, 
                                    columns = ['Number of Accepted Reads',
                                               'Mean Quality Score'])
    rd_qual_discarded = pd.DataFrame(index = list_of_samps, 
                                     columns = ['Number of Discarded Reads', 
                                                'Mean Quality Score'])
    
    return bc_counts, bc_recovery, rd_qual_accepted, rd_qual_discarded 

In [ ]:
# Define a function that counts barcodes and updates a dictionary with the resulting counts

# FUNCTION PARAMETERS
# lib_bc_list: list of the library member barcode sequences
# list_of_samps: list of the samples in the sequencing run 
# fastq_paths: list of path objects that link to the FASTQ files
# qual_score_cutoff: int that designates the average quality score under which a read
#                    should be removed from downstream analyses; input -1 to use all reads 
# counts_df: an empty dataframe generated by the previous cell that will hold the barcode counts 
# rec_df: an empty dataframe generated by the previous cell that will hold the barcode recovery info
# qual_acc_df: an empty dataframe generated by the previous cell that will hold the total number 
#              and mean quality scores of reads that meet the qual_score_cutoff
# qual_dis_df: an empty dataframe generated by the previous cell that will hold the total number 
#              and mean quality scores of reads that do not meet the qual_score_cutoff
# file_start: int that designates the index of the file to start analyzing, typically 0

def count_barcodes(lib_bc_list, list_of_samps, fastq_paths, 
                   qual_score_cutoff, 
                   counts_df, rec_df, qual_acc_df, qual_dis_df,
                   file_start): 
    
    ## SET UP FILE AND INDEX TRACKERS 
    
    # Define a current file index
    current_file_index = file_start
    
    
    ## CREATE A DICTIONARY TO STORE BARCODE COUNTS 

    # Set all keys to the barcodes in the library and all values to 0
    # Note: barcodes are obtained from 'Barcode Sequence (Reverse)' because Read 2 is being analyzed.  
    bc_dict = dict.fromkeys(bc_df['Barcode Sequence (Reverse)'], 0)
    
    
    ## OBTAIN THE BARCODES FROM EACH FILE 

    # Loop through each of the files 
    for current_file in fastq_paths[file_start:]:
        
        # Check that files are loading 
        print(f'Checking File = {current_file_index}')

        # Create a read counter 
        num_reads = 0 

        # Create a barcode counter 
        num_bcs_recovered = 0
            
        # Create read counters for the quality score metrics
        num_accepted_reads = 0
        num_discarded_reads = 0
            
        # Create a list to store the mean read quality scores
        # Note: these will be averaged later. 
        mean_qual_score_accepted_reads = []
        mean_qual_score_discarded_reads = []
            

        ## PARSE THE READS
    
        # Create a sequence parser 
        seq_parser = SeqIO.parse(current_file, "fastq")
        
        # Iterate through the parser 
        while True: 
            
            # Try to move to the next record 
            try: 
                
                # Choose the next record
                rec = next(seq_parser)
                
                # Add one to the read counter 
                num_reads += 1
                
                # Check if read is high-quality 
                if qual_score_cutoff > 0: 
                    
                    # Obtain the Phred quality scores for the read 
                    qual_scores = rec.letter_annotations['phred_quality']

                    # Calculate the mean quality score for the read 
                    mean_qual_score = np.mean(qual_scores)
                    
                    # If the mean quality score for the read is 
                    # less than the quality score cutoff, 
                    # move to the next read in the file; 
                    # otherwise, go through the rest of code in the for loop.
                    if mean_qual_score >= qual_score_cutoff:
                        mean_qual_score_accepted_reads.append(mean_qual_score)
                        num_accepted_reads += 1 
                    else: 
                        mean_qual_score_discarded_reads.append(mean_qual_score)       
                        num_discarded_reads += 1   
                        continue 


                # Obtain just the sequence from the read 
                current_seq = rec.seq

                # Obtain the barcode from the barcode-containing region of the read 
                # Note: this is the location for Read 2.
                read_bc = current_seq[46:54]

                # Check if any barcode in the library matches the read sequence 
                if read_bc in bc_dict:
                    
                    # Update the dictionary entry for that barcode
                    bc_dict[read_bc] = bc_dict.get(read_bc) + 1
                    
                    # Add one to the barcode counter 
                    num_bcs_recovered += 1
                
            # Stop the while loop once there is nothing "next" in the parser 
            # (i.e., once the last read in the file has been analyzed)
            except StopIteration:
                break
            # Print the error and move to the next record
            except ValueError as e:
                print(f'ValueError: {e}')
                continue    
                
                
        # Obtain the sample name to label columns in the dataframes
        current_samp_name = list_of_samps[current_file_index]
                
        # Add the dictionary for the current sample to the barcode counts dataframe 
        counts_to_add_df = pd.DataFrame.from_dict(bc_dict, 
                                                  orient = 'index',
                                                  columns = [current_samp_name])
        
        counts_df[current_samp_name] = counts_to_add_df[current_samp_name]
        
        
        
        # Add the read and barcode recovery info to the corresponding dataframes 
        rec_df.iloc[current_file_index, 0] = num_reads
        rec_df.iloc[current_file_index, 1] = num_bcs_recovered
        rec_df.iloc[current_file_index, 2] = round(num_bcs_recovered/num_reads*100,2)
        
        # Add the quality score info to the corresponding dataframes 
        qual_acc_df.iloc[current_file_index, 0] = num_accepted_reads
        qual_acc_df.iloc[current_file_index, 1] = np.mean(mean_qual_score_accepted_reads)
        
        qual_dis_df.iloc[current_file_index, 0] = num_discarded_reads
        qual_dis_df.iloc[current_file_index, 1] = np.mean(mean_qual_score_discarded_reads)
        
            
        # Update the current file index 
        current_file_index += 1   
                
        
        # Reset the counts in the dictionary for the next sample
        bc_dict =  {x:0 for x in bc_dict}

In [ ]:
# Create dataframes for storing barcode counts and barcode recovery information 

# Note: 'Sequencing_Run_Title' should be replaced with the column header of the sequencing run of interest
#       from the Excel file with the sample lists. 

counts, rec, acc, dis = make_bc_dfs(list_of_samps = samp_df['Sequencing_Run_Title'], 
                                    lib_bc_list = list(bc_df['Barcode Sequence (Reverse)']))

In [ ]:
# Count the library member barcodes in the specified FASTQ files

# Note: 'Sequencing_Run_Title' should be replaced with the column header of the sequencing run of interest
#       from the Excel file with the sample lists (Samples List.xlsx > samp_df). 

# Note: corresponding FASTQ files and sample names must be indexed identically before being passed 
#       as arguments to the parameters "fastq_paths" and "list_of_samps", respectively. 
#       Ordering the files and sample names differently will lead to incorrect 
#       barcode count assignments for the samples. 

count_barcodes(lib_bc_list = list(bc_df['Barcode Sequence (Reverse)']), # Note: this checks for barcodes in Read 2. 
               list_of_samps = samp_df['Sequencing_Run_Title'],
               fastq_paths = fastq_paths_r2, 
               qual_score_cutoff = 20, # Note: this indicates a Phred score cutoff of 20 (99% base call accuracy). 
               counts_df = counts, 
               rec_df = rec, 
               qual_acc_df = acc, 
               qual_dis_df = dis,
               file_start = 0) # Note: this starts at the first file in the list. 
                               #       File subsets can be analyzed by changing this argument; 
                               #       in that case, list_of_samps must also be updated to select 
                               #       the correct set of sample names for the corresponding FASTQ files. 

# Add library member information to the barcode count dataframe

In [ ]:
# Check that the order of the barcodes remained the same 
print(np.unique(bc_df['Barcode Sequence (Reverse)'] == counts.index))

# Note: the output should be "True". 

In [ ]:
# Replace the indices in the barcode count dataframe with the library member names
counts.index = list(bc_df['Library Member Name'])

In [ ]:
# Print out the list of library members 
print(list(bc_df['Library Member Name']))
print(len(bc_df['Library Member Name']))

In [ ]:
# Create an ordered list of library members 
ordered_list_of_lib_mems = ['Wuhan Strain', 'D614G', 
                            'B.1.1.277', 'B.1.1.298', 'B.1.1.302', 
                            'B.1.1.7',
                            'B.1.160',
                            'B.1.221',
                            'B.1.258', 
                            'B.1.351 v1', 'B.1.351 v2', 'B.1.351 v3', 'B.1.367',
                            'B.1.429',
                            'B.1.525', 'B.1.526', 'B.1.526.1', 'B.1.526.2',
                            'B.1.617',
                            'P.1', 'P.2',
                            'SARS 2 N CTD only', 'SARS 2 E', 'SARS 2 M',
                            'SARS-CoV-1', 'MERS',  
                            'WIV1-CoV', 'MHV',
                            'OC43', '229E', 'HKU1', 'NL63', 
                            'H1', 'H3', 'N1', 'N2',
                            'RSV F',
                            'Measles H', 'Measles F',
                            'CMV gB', 'EBV gB',
                            'HIV  SOSIP Pstalk', 
                            'Nipah Virus G', 
                            'Dengue Virus Type 2 Monomer', 'Dengue Virus Type 2 Dimer',
                            '10% Ladder','1% Ladder','0.1% Ladder']

In [ ]:
# Reorder the rows of the dataframes 
counts = counts.reindex(ordered_list_of_lib_mems)

In [ ]:
# Write the dataframes to files 
# Note: replace 'DATE SEQUENCING_RUN_NAME' with the date of file creation 
#       and a description of the sequencing run
counts.to_csv('DATE SEQUENCING_RUN_NAME Barcode Counts.csv')
rec.to_csv('DATE SEQUENCING_RUN_NAME Barcode Recovery.csv')
acc.to_csv('DATE SEQUENCING_RUN_NAME Accepted Reads.csv')
dis.to_csv('DATE SEQUENCING_RUN_NAME Discarded Reads.csv')

# Combine data from multiple replicates/lanes

If processing multiple replicates/lanes of data for the same samples, make sure to sum corresponding barcode counts before proceeding. The following cells provide examples of how to combine barcode counts from different replicates/lanes and update the barcode recovery metrics. 

The example provides code for four lanes/replicates. When generating the dataframes above, name them clearly to indicate which dataframe corresponds to which lane/replicate. 

In [ ]:
# Add barcode count dataframes together in an element-wise fashion 
total_counts = counts_1 + counts_2 + counts_3 + counts_4

In [ ]:
# Make a dataframe to hold the total recovery stats
total_rec = pd.DataFrame(index = rec_1.index, 
                         columns = rec_1.columns)

total_rec['Number of Reads per Sample'] = rec_1['Number of Reads per Sample'] + \
                                          rec_2['Number of Reads per Sample'] + \
                                          rec_3['Number of Reads per Sample'] + \
                                          rec_4['Number of Reads per Sample']

total_rec['Number of Barcodes Recovered per Sample'] = rec_1['Number of Barcodes Recovered per Sample'] + \
                                                       rec_2['Number of Barcodes Recovered per Sample'] + \
                                                       rec_3['Number of Barcodes Recovered per Sample'] + \
                                                       rec_4['Number of Barcodes Recovered per Sample']
                
total_rec['Percent Barcode Recovery'] = round(total_rec['Number of Barcodes Recovered per Sample']/total_rec['Number of Reads per Sample']*100,2)

In [ ]:
# Make a dataframe to hold the total accepted reads stats 
total_acc = pd.DataFrame(index = acc_1.index, 
                         columns = acc_1.columns)

total_acc['Number of Accepted Reads'] = acc_1['Number of Accepted Reads'] + \
                                        acc_2['Number of Accepted Reads'] + \
                                        acc_3['Number of Accepted Reads'] + \
                                        acc_4['Number of Accepted Reads'] 

total_acc['Mean Quality Score'] = pd.concat([acc_1['Mean Quality Score'], acc_2['Mean Quality Score'],
                                             acc_3['Mean Quality Score'], acc_4[['Mean Quality Score']]],
                                            axis = 1).mean(axis = 1)

In [ ]:
# Make a dataframe to hold the total discarded reads stats 
total_dis = pd.DataFrame(index = dis_1.index, 
                         columns = dis_1.columns)

total_dis['Number of Discarded Reads'] = dis_1['Number of Discarded Reads'] + \
                                         dis_2['Number of Discarded Reads'] + \
                                         dis_3['Number of Discarded Reads'] + \
                                         dis_4['Number of Discarded Reads'] 

total_dis['Mean Quality Score'] = pd.concat([dis_1['Mean Quality Score'], dis_2['Mean Quality Score'],
                                             dis_3['Mean Quality Score'], dis_4[['Mean Quality Score']]],
                                            axis = 1).mean(axis = 1)

In [ ]:
# Write all the total dataframes to csv files
total_counts.to_csv('DATE SEQUENCING_RUN_NAME Total Barcode Counts.csv')
total_rec.to_csv('DATE SEQUENCING_RUN_NAME Total Barcode Recovery.csv')
total_acc.to_csv('DATE SEQUENCING_RUN_NAME Total Accepted Reads.csv')
total_dis.to_csv('DATE SEQUENCING_RUN_NAME Total Discarded Reads.csv')

# Visualize barcode recovery metrics 

#### Graph read counts, barcode counts, and barcode recovery across samples and library members

In [ ]:
# Define a function to report stats on the barcodes recovered

# FUNCTION PARAMETERS
# rec_df: dataframe with barcode recovery information 
# num_bins: int with number of bins for the histogram

def barcode_recovery(rec_df, num_bins): 
    
    # Calculate average and median numbers of reads per sample
    avg_reads = rec_df['Number of Reads per Sample'].mean()
    med_reads = np.median(rec_df['Number of Reads per Sample'])
    
    # Plot reads per sample
    sns.histplot(rec_df['Number of Reads per Sample'], bins = num_bins, color = 'lime')
    plt.axvline(avg_reads, color = 'darkgreen', linestyle = 'dashed', linewidth = 1)
    plt.axvline(med_reads, color = 'forestgreen', linestyle = 'dotted', linewidth = 1)
    plt.xlabel('Number of Reads')
    plt.ylabel('Number of Samples')
    plt.title(f'Reads per Sample (Mean: {avg_reads:.2e} | Median: {med_reads:.2e})')
    plt.show()
    
    # Calculate average and median numbers of barcodes recovered per sample
    avg_bcs = rec_df['Number of Barcodes Recovered per Sample'].mean()
    med_bcs = np.median(rec_df['Number of Barcodes Recovered per Sample'])
    
    # Plot barcodes recovered per sample
    sns.histplot(rec_df['Number of Barcodes Recovered per Sample'], bins = num_bins, color = 'turquoise')
    plt.axvline(avg_bcs, color = 'darkturquoise', linestyle = 'dashed', linewidth = 1)
    plt.axvline(med_bcs, color = 'mediumturquoise', linestyle = 'dotted', linewidth = 1)
    plt.xlabel('Number of Barcodes Recovered')
    plt.ylabel('Number of Samples')
    plt.title(f'Barcodes Recovered per Sample (Mean: {avg_bcs:.2e} | Median: {med_bcs:.2e})')
    plt.show()
    
    # Calculate average and median percentages of reads with barcodes per sample
    avg_rec = rec_df['Percent Barcode Recovery'].mean()
    med_rec = np.median(rec_df['Percent Barcode Recovery'])
    
    # Plot percent barcode recovery per sample 
    sns.histplot(rec_df['Percent Barcode Recovery'], bins = num_bins, color = 'orchid')
    plt.axvline(avg_rec, color = 'darkorchid', linestyle = 'dashed', linewidth = 1)
    plt.axvline(med_rec, color = 'mediumorchid', linestyle = 'dotted', linewidth = 1)
    plt.xlabel('Percent of Reads with Barcodes')
    plt.ylabel('Number of Samples')
    plt.title(f'Barcode Recovery Rates (Mean: {avg_rec:.1f}% | Median: {med_rec:.1f}%)')
    plt.show()

In [ ]:
# Plot the recovery stats
barcode_recovery(rec, 15)

#### Graph the barcode counts for each library member 

In [ ]:
# Sum the number of barcodes counted for each library member 
bc_lib_mem_sums = pd.DataFrame(counts.sum(axis = 1), 
                               columns = ['Total Barcode Count'])

In [ ]:
# Graph the library member sums 

# Set figure size 
plt.figure(figsize = (70,15))

# Create a range for the x-axis
x_axis_lib_mem_sums = np.arange(len(bc_lib_mem_sums.index)) 
  
# Create the bars in the plot 
plt.bar(x_axis_lib_mem_sums, bc_lib_mem_sums['Total Barcode Count'], 
        1, edgecolor = 'black') 

# Format the plot 
plt.xticks(x_axis_lib_mem_sums, bc_lib_mem_sums.index, 
           rotation = 30, ha = 'right', fontsize = 30) 
plt.yticks(fontsize = 30)
plt.xlabel("Library Members", fontsize = 50) 
plt.ylabel("Barcode Count", fontsize = 50, labelpad = 20) 
plt.title("Barcode Count Recorded for Each Library Member", fontsize = 60) 
plt.legend(fontsize = 40) 
plt.show() 

#### Graph the read and barcode recovery data by sample

In [ ]:
# Define a function to plot different barcode recovery stats by sample

# FUNCTION PARAMETERS 
# data_to_plot: list of 1D dataframes 
# bar_color: color for the bars
# fig_w: int specifying the width of the figure
# fig_h: int specifying the height of the figure
# x_label: str with label for x-axis
# y_label: str with label for y-axis
# title: str with title for the plot
# value_to_highlight: value in scientific notation at which to plot a horizontal line 
# y_min: minimum y-axis value 
# y_max: maximum y-axis value 

def plot_stats_by_sample(data_to_plot,
                         bar_color,
                         fig_w, fig_h,
                         x_label, y_label, title,
                         value_to_highlight = None,
                         y_min = None, y_max = None): 
    
    # Set figure size 
    plt.figure(figsize = (fig_w, fig_h))
    
    # Create a range for the x-axis 
    x_axis_samp_sums = np.arange(96) # Note: this assumes that the dataframes contain 96 samples each.  
    
    # Create the bars in the plot 
    plt.bar(x_axis_samp_sums, data_to_plot, 
            color = bar_color, edgecolor = 'black', linewidth = 2) 
  
    # Format the plot 
    plt.xticks(x_axis_samp_sums, data_to_plot.index, 
               rotation = 30, ha = 'right', fontsize = 35) 
    plt.yticks(fontsize = 35)
    plt.xlabel(x_label, fontsize = 45) 
    plt.ylabel(y_label, fontsize = 45) 
    plt.ylim(bottom = y_min, top = y_max)
    plt.title(title, fontsize = 60) 

    # Add the horizontal line 
    if value_to_highlight != None: 
        plt.axhline(y = value_to_highlight, color = 'black', 
                    linestyle = 'dashed', linewidth = 5) 

    # Add horizontal grid lines 
    plt.grid(axis = 'y', color = 'grey')
    ax = plt.gca()
    ax.set_axisbelow(True)
    
    plt.show()

In [ ]:
# Plot the number of reads per sample 
plot_stats_by_sample(data_to_plot = rec['Number of Reads per Sample'],
                     bar_color = 'lime',
                     fig_w = 150, fig_h = 15,
                     x_label = 'Sample', 
                     y_label = 'Number of Reads', 
                     title = 'Number of Reads per Sample',
                     y_max = 0.8e6) # Note: change this value for the data being analyzed. 

In [ ]:
# Plot the number of barcodes per sample
plot_stats_by_sample(data_to_plot = rec['Number of Barcodes Recovered per Sample'],
                     bar_color = 'turquoise',
                     fig_w = 150, fig_h = 15,
                     x_label = 'Sample', 
                     y_label = 'Number of Barcodes Recovered', 
                     title = 'Number of Barcodes Recovered per Sample',
                     y_max = 0.5e6) # Note: change this value for the data being analyzed. 

In [ ]:
# Plot the barcode recovery rate for each sample
plot_stats_by_sample(data_to_plot = rec['Percent Barcode Recovery'],
                     bar_color = 'orchid',
                     fig_w = 150, fig_h = 15,
                     x_label = 'Sample', 
                     y_label = '% Reads with Barcodes Recovered', 
                     title = 'Barcode Recovery per Sample',
                     y_max = 100)

#### Graph the numbers of accepted vs discarded reads 

In [ ]:
# Define a function to graph accepted and discarded read counts 

# FUNCTION PARAMETERS
# acc_df: dataframe with counts of accepted reads 
# dis_df: dataframe with counts of discarded reads
# fig_w: int specifying the width of the figure
# fig_h: int specifying the height of the figure
# title: str with title for the plot 
# y_max: maximum y-axis value 
# y_min: minimum y-axis value 


def plot_acc_dis_reads(acc_df, dis_df, 
                       fig_w, fig_h, 
                       title,
                       y_max = None, y_min = None): 

    # Create a range for the x-axis
    x_axis = np.arange(96) # Note: this assumes that the dataframes contain 96 samples each.  

    # Set the figure size 
    plt.figure(figsize = (fig_w,fig_h))

    # Create the bars in the plot 
    plt.bar(x_axis, acc_df['Number of Accepted Reads'], 
            label = 'Accepted', color = 'gray') 
    plt.bar(x_axis, dis_df['Number of Discarded Reads'], 
            label = 'Discarded', color = 'orange',
            bottom = acc_df['Number of Accepted Reads']) 

    # Format the plot 
    plt.xticks(x_axis, acc_df.index, 
               rotation = 30, ha = 'right', fontsize = 35) 
    plt.yticks(fontsize = 35)
    plt.xlabel("Samples", fontsize = 45, labelpad = 30) 
    plt.ylabel("Number of Reads", fontsize = 45, labelpad = 30) 
    plt.title(title, fontsize = 60, pad = 30) 
    plt.ylim(bottom = y_min, top = y_max)
    plt.legend(fontsize = 50, loc = 'upper right') 

    plt.show()

In [ ]:
# Plot the numbers of accepted and discarded reads 
plot_acc_dis_reads(acc_df = acc, 
                   dis_df = dis, 
                   fig_w = 150, fig_h = 15, 
                   title = 'Accepted and Discarded Reads',
                   y_max = 0.8e6) # Note: change this value for the data being analyzed. 

#### Graph the quality scores for the accepted and discarded reads

In [ ]:
# Define a function to graph the quality score data 

# FUNCTION PARAMETERS
# acc_df: dataframe with counts of accepted reads 
# dis_df: dataframe with counts of discarded reads
# fig_w: int specifying the width of the figure
# fig_h: int specifying the height of the figure
# title: str with title for the plot 
# y_max: maximum y-axis value 
# y_min: minimum y-axis value 

def plot_q_scores(acc_df, dis_df, 
                  fig_w, fig_h, 
                  title, y_max = None, y_min = None):

    # Create a range for the x-axis
    x_axis = np.arange(96) # Note: this assumes that the dataframes contain 96 samples each.  

    # Set the figure size 
    plt.figure(figsize = (fig_w,fig_h))

    # Create the bars in the plot 
    plt.bar(x_axis + 0, acc_df['Mean Quality Score'], 
            0.2, label = 'Accepted', color = 'gray') 
    plt.bar(x_axis + 0.2, dis_df['Mean Quality Score'], 
            0.2, label = 'Discarded', color = 'orange')

    # Format the plot 
    plt.xticks(x_axis, acc_df.index, 
               rotation = 30, ha = 'right', fontsize = 30) 
    plt.yticks(fontsize = 30)
    plt.xlabel("Samples", fontsize = 45, labelpad = 30) 
    plt.ylabel("Mean Quality Score", fontsize = 45, labelpad = 30) 
    plt.title(title, fontsize = 60, pad = 30)
    plt.ylim(bottom = y_min, top = y_max)
    plt.legend(fontsize = 30, loc = 'upper right') 

    plt.show()

In [ ]:
# Create plots for the quality score info
plot_q_scores(acc_df = acc, 
              dis_df = dis, 
              fig_w = 150, 
              fig_h = 15, 
              title = 'Mean Quality Scores for Accepted and Discarded Reads',
              y_max = 40)

# Determine which ladder virus to use for normalization

In [ ]:
# Define a function to graph ladder virus barcode counts

# FUNCTION PARAMETERS
# bc_counts_df: dataframe with the barcode counts 

def plot_ladders(bc_counts_df): 
    
    # Set the figure size 
    plt.figure(figsize = (150,15))

    # Create a range for the x-axis
    x_axis_ladder = np.arange(96) # Note: this assumes that the dataframes contain 96 samples each.  

    # Create the bars in the plot 
    plt.bar(x_axis_ladder - 0.3, bc_counts_df.loc['10% Ladder'], 
            0.2, label = '10% Ladder', color = '#C5E898', edgecolor = 'black') 
    plt.bar(x_axis_ladder - 0.1, bc_counts_df.loc['1% Ladder'], 
            0.2, label = '1% Ladder', color = '#29ADB2', edgecolor = 'black') 
    plt.bar(x_axis_ladder + 0.1, bc_counts_df.loc['0.1% Ladder'], 
            0.2, label = '0.1% Ladder', color = '#0766AD', edgecolor = 'black') 

    # Format the plot 
    plt.xticks(x_axis_ladder, bc_counts_df.columns, rotation = 30, 
               ha = 'right', fontsize = 30) 
    plt.yticks(fontsize = 30)
    plt.xlabel("Samples", fontsize = 40, labelpad = 30) 
    plt.ylabel("Ladder Virus Barcode Count", fontsize = 40, labelpad = 30) 
    plt.legend(fontsize = 40, loc = 'upper right') 

    # Calculate the mean of all the barcode counts (excluding the ladder viruses' barcode counts)
    mean_bc_count = bc_counts_df.iloc[:-3,:].mean().mean()

    # Add the horizontal line 
    plt.axhline(y = mean_bc_count, color = 'black', linestyle = 'dashed', linewidth = 5) 

    plt.title(f"Ladder Virus Barcode Counts for Each Sample - Mean Non-Ladder Library Member Barcode Count: {mean_bc_count:.0f}", 
              fontsize = 50, pad = 40) 
    
    plt.show()

In [ ]:
# Create the plot of the ladder virus barcode counts
plot_ladders(counts)

# Use the 1% ladder for ARCADE samples 
# Use the 10% ladder for the neutralization assay samples 

In [ ]:
# Define a function to determine how many samples 
# have ladder virus barcode counts below a certain threshold 

# FUNCTION PARAMETERS
# bc_counts_df: dataframe containing the ladder virus barcode counts 
# thresholds: list with minimum barcode count (int) acceptable for each ladder 
# plate_run_desc: description of the plate/run 

def ladder_threshold(bc_counts_df, thresholds, plate_run_desc): 
    
    # Create a list of the ladder virus labels matching the labels in the dataframe
    ladders = ['10% Ladder', '1% Ladder', '0.1% Ladder']
    
    # Loop through all the ladder viruses
    for lad in range(len(ladders)): 
        
        # Print a header for the current ladder virus being checked
        print(f'{plate_run_desc} samples with low {ladders[lad]} barcode counts')
        
        # Loop through all the samples 
        for samp in range(len(bc_counts_df.columns)): 

            # Obtain the ladder virus barcode count for the current sample 
            samp_lad_count = bc_counts_df.loc[ladders[lad]][samp]

            # Check if the ladder virus barcode count for the sample is below the acceptable threshold
            if samp_lad_count < thresholds[lad]: 

                # Print the names of samples with low ladder virus barcode counts 
                print(f'Sample: {bc_counts_df.columns[samp]} | Barcode Count: {samp_lad_count}')
                
        print('\n')

In [ ]:
# Report samples with low ladder virus barcode counts to determine which ladder virus to use for normalization 
ladder_threshold(bc_counts_df = counts, 
                 thresholds = [500, 50, 5],
                 plate_run_desc = "Example Run")

# 10% Ladder
# - Usually only samples that did not receive the ladder viruses have low barcode counts for this ladder virus
# - Barcode counts for this ladder virus tend to be very elevated compared to the library member barcode counts

# 1% Ladder
# - Some neutralization assay samples have low barcode counts even for this ladder virus 

# 0.1% Ladder 
# - Most ARCADE and neutralization assay samples have low barcode counts for this ladder virus 

# Use the 10% Ladder for the neutralization assay samples
# Use the 1% Ladder for the ARCADE samples

# Normalize the library member barcode counts for each sample using the selected ladder virus's barcode counts for each sample

In [ ]:
# Define a function to normalize barcode counts using a ladder virus's barcode counts

# FUNCTION PARAMETERS
# bc_counts_df: dataframe containing the barcode counts for all samples and library members and ladder viruses
# ladder_to_use: str with the name of the ladder to use 
# plate_run_desc: str with a description of the plate/run 
# date: str with the date of data normalization 

def normalize_bc_counts(bc_counts_df, ladder_to_use, plate_run_desc, date): 
    
    # Create an empty dataframe to store the normalized counts
    counts_norm = pd.DataFrame(index = bc_counts_df.index, 
                               columns = bc_counts_df.columns)

    # Loop through the samples in the dataframe 
    for samp in range(len(bc_counts_df.columns)): 

        # Obtain the ladder virus barcode count from the current sample
        ladder_divisor = bc_counts_df.loc[ladder_to_use][samp]

        # Make sure the current ladder virus barcode count is not 0
        if ladder_divisor > 0: 

            # Loop through the library members in the dataframe 
            for mem in range(len(bc_counts_df.index)):

                # Record the rounded normalized barcode counts
                counts_norm.iloc[mem, samp] = round(bc_counts_df.iloc[mem, samp]/ladder_divisor, 2)

            # Note: any samples with a ladder virus barcode count of 0 will have NaN in the corresponding 
            #       locations in the dataframe
        
        # Report samples with ladder virus barcode counts of 0 
        else: 
            print('Sample', bc_counts_df.columns[samp], 'has a', 
                  ladder_to_use, 'barcode count of', ladder_divisor)
            
    # Write normalized counts to a CSV file 
    counts_norm.to_csv(f'{date} {plate_run_desc} - Ladder-Normalized Barcode Counts.csv')
    
    return counts_norm

In [ ]:
# Normalize library member barcode counts by the ladder virus barcode counts 
norm_counts = normalize_bc_counts(bc_counts_df = counts, 
                                  ladder_to_use = "1% Ladder", # Note: change this selection based on the above analysis. 
                                  plate_run_desc = "Example ARCADE Assay", # Note: change this for the data being analyzed. 
                                  date = "20251231") # Note: change this based on the date normalization is performed.

# Note: samples that were not provided ladder virus should report ladder virus barcode counts of 0.  

At this point, samples corresponding to flow cytometry controls (e.g., NT = Not Transduced) and library members corresponding to the ladder viruses can be removed from the dataframe. The following cell provides example code that can perform these removal steps. 

In [ ]:
# Remove the flow cytometry controls (e.g., NT) and ladders from the ladder-normalized barcode counts dataframe 

# Obtain the samples to remove corresponding to the flow cytometry controls 
samples_to_remove = []

# Loop through the columns in the dataframe to find the samples to remove 
for col in range(len(norm_counts.columns)): 

    if "NT" in norm_counts.columns[col]: 

        samples_to_remove.append(norm_counts.columns[col])

    elif "CD-594-Ctrl" in norm_counts.columns[col]: 

        samples_to_remove.append(norm_counts.columns[col])

# Print the samples being removed 
print(samples_to_remove)

# Remove the columns with the flow cytometry controls
norm_counts.drop(labels = samples_to_remove, # Note: change this based on the samples in the sample list. 
                 axis = 1, 
                 inplace = True)

# Remove the rows with the ladder virus values 
norm_counts.drop(labels = ['10% Ladder', '1% Ladder', '0.1% Ladder'],
                 axis = 0, 
                 inplace = True)

# Calculate averages and standard deviations across replicates 

In the ARCADE paper, ladder-normalized barcode counts are referred to as "ladder-normalized barcode fractions." The variable "norm_counts" in the following cells contains these "ladder-normalized barcode fractions." 

In [ ]:
# Obtain the names of the samples in the ladder-normalized barcode counts dataframe 

# Create a list for storing the names 
samp_list = []

# Loop through each sample
for samp in norm_counts.columns: 
    
    # Remove the replicate number from the sample 
    samp_name = samp_name[:-2]
    
    # Add the sample name to the list if it hasn't been added yet
    if samp_name not in samp_list: 
        samp_list.append(samp_name)

# Observe the list 
print(samp_list)
print(len(samp_list))

In [ ]:
# Create a dataframe for storing the averages of the replicates
norm_avgs = pd.DataFrame(0, 
                         index = norm_counts.index, 
                         columns = samp_list)

# Create a dataframe for storing the standard deviations of the replicates 
norm_stds = pd.DataFrame(0, 
                         index = norm_counts.index, 
                         columns = samp_list)

In [ ]:
# Compute averages and standard deviations and store them in the appropriate location in the dataframe 

# Loop through the rows in the dataframe with the sample library member barcode counts
for row in range(norm_counts.shape[0]): 

    # Loop through the sample names 
    for col in range(len(samp_list)):
        
        # Obtain the values to average (every three columns)
        start_val = col*3
        end_val = col*3 + 3
        vals_to_use = norm_counts.iloc[row, start_val:end_val]
        
        # Add average value
        norm_avgs.iloc[row, col] = round(vals_to_use.mean(), 2)
        
        # Add standard deviation value 
        norm_stds.iloc[row, col] = round(vals_to_use.std(), 2)

In [ ]:
# Save the dataframes to files 
norm_avgs.to_csv('DATE SEQUENCING_RUN_NAME 1% Ladder-Normalized Barcode Fractions - Averages.csv')
norm_stds.to_csv('DATE SEQUENCING_RUN_NAME 1% Ladder-Normalized Barcode Fractions - Standard Deviations.csv')

# Note: change 'DATE SEQUENCING_RUN_NAME' based on the experiment being analyzed. 
# Note: change the ladder virus listed if not using the 1% ladder. 

# Perform double normalization using the No Serum/Virus Only samples

Note: for the neutralization assay, samples labeled VO (Virus Only) were used for normalization. These samples are conceptually the same as the ARCADE No Serum samples, but they utilize the lentiviral library doses, cell line, and incubation time for the neutralization assay.  

In [ ]:
# Create a list of the No Serum samples' sample names

# Note: ensure these names match those in the norm_avgs and norm_stds dataframes
ns_samp_names = ['No Serum 1', 'No Serum 2', 'No Serum 3',
                 'No Serum 4', 'No Serum 5', 'No Serum 6', 
                 'No Serum 7']

In [ ]:
# Create a dataframe for storing the pooled standard deviations of the No Serum samples' 
# ladder-normalized barcode fractions for each library member
no_serum_avg_std = pd.DataFrame(index = norm_avgs.index,
                                columns = ['No Serum Pooled Avg', 'No Serum Pooled SD'])

In [ ]:
# Average all No Serum samples' average ladder-normalized barcode fractions

# Note: this is averaging across sample columns. 
no_serum_avg_std['No Serum Pooled Avg'] = norm_avgs.loc[:,ns_samp_names].mean(axis = 1).round(2)

In [ ]:
# Compute the pooled standard deviations for the No Serum samples 

# Note: this is only necessary if using No Serum sample replicates across plates for normalization. 
#       The ARCADE data on the SRA contains No Serum samples from 7 separate plates 
#       with 3 replicates per plate. The neutralization assay data on the SRA, however, 
#       contains No Serum/Virus Only samples for three different lentiviral library doses 
#       with three replicates per sample from a single plate. A simple standard deviation 
#       can be used instead of a pooled standard deviation if the replicates being averaged 
#       are not grouped in any way (i.e., not belonging to different plates). 

# Loop through each library member 
for mem in no_serum_avg_std.index: 
    
    # Calculate the pooled standard deviation 
    
    # Set the squared standard deviation sum to zero 
    sd_sq_sum = 0
    
    # Set the number of elements to zero (it should always end up as 7)
    k = 0

    # Loop through each No Serum sample 
    for samp in ns_samp_names: 

        # Calculate the square of the standard deviation 
        sd_sq_sum = sd_sq_sum + norm_stds.loc[mem,samp]**2
        k = k + 1
    
    no_serum_avg_std.loc[mem, 'No Serum Pooled SD'] = math.sqrt(sd_sq_sum / k)

In [ ]:
# Plot the No Serum samples' average ladder-normalized barcode fractions with standard deviations as error bars 

# Create a figure 
plt.figure(figsize = (50, 10))
plt.grid(axis = 'y', color = 'grey')

# Create a list of colors to categorize the library members 
color_list = ['#8CFF00']*21 + ['#E6BD00']*3 + ['#E52EB5']*8 + ['#00ADC3']*4 + ['#BDBCBC']*9

# Produce the plot 
plt.bar(no_serum_avg_std.index,
        no_serum_avg_std['No Serum Pooled Avg'],
        yerr = no_serum_avg_std['No Serum Pooled SD'],
        edgecolor = 'black',
        color = color_list, 
        linewidth = 4, 
        error_kw = dict(lw = 5))

# Create a legend 
no_serum_legend = [mpatches.Patch(facecolor = '#8CFF00',
                                  label = 'SARS-CoV-2 Spike Variants',
                                  edgecolor = 'black',
                                  linewidth = 2),
                   mpatches.Patch(facecolor = '#E6BD00',
                                  label = 'SARS-CoV-2 Non-Spike Proteins',
                                  edgecolor = 'black',
                                  linewidth = 2),
                   mpatches.Patch(facecolor = '#E52EB5',
                                  label = 'Additional Coronavirus Proteins',
                                  edgecolor = 'black',
                                  linewidth = 2),
                   mpatches.Patch(facecolor = '#00ADC3',
                                  label = 'Influenza Envelope Proteins',
                                  edgecolor = 'black',
                                  linewidth = 2),
                   mpatches.Patch(facecolor = '#BDBCBC',
                                  label = 'Other Viral Proteins',
                                  edgecolor = 'black',
                                  linewidth = 2)]

# Format the plot
plt.xticks(rotation = 30, ha = 'right', fontsize = 25)
plt.yticks(fontsize = 25)
plt.xlabel('Library Members', fontsize = 35)
plt.ylabel('Ladder-Normalized \nBarcode Fraction', fontsize = 35, labelpad = 20)
plot_title = 'Pooled Average Ladder-Normalized Barcode Fractions for the No Serum Samples\n'
plt.title(plot_title, fontsize = 50, pad = 20)

plt.legend(handles = no_serum_legend, bbox_to_anchor = (0.995, 1.15),
           fontsize = 30, frameon = True, ncol = 5, edgecolor = 'grey')

In [ ]:
# Divide each sample's average library member ladder-normalized barcode fractions 
# by the corresponding No Serum samples' average library member ladder-normalized barcode fractions
avgs_no_serum_norm = norm_avgs.div(no_serum_avg_std['No Serum Pooled Avg'], axis = 'index').round(2)

In [ ]:
# Perform error propagation to calculate the standard deviations of the double-normalized barcode fractions

# Define a function to perform the error propagation 

# FUNCTION PARAMETERS
# avgs_df: dataframe with average ladder-normalized barcode fractions for all samples
# stds_df: dataframe with standard deviations of ladder-normalized barcode fractions for all samples
# no_serum_df: dataframe with average and standard deviations of ladder-normalized barcode fractions 
#              for only the No Serum samples
# no_serum_norm_avg_df: dataframe with average sample ladder-normalized barcode fractions divided 
#                       by average No Serum ladder-normalized barcode fractions 

# Note: avgs_df, stds_df, and no_serum_norm_avg_df must have the same dimensions and orientation
# Note: no_serum_df must have the same number and order of rows as the others

def calc_no_serum_error_prop(avgs_df, stds_df, 
                             no_serum_df, no_serum_norm_avg_df): 

    # Create a new dataframe to return 
    no_serum_stds = pd.DataFrame(index = stds_df.index, 
                                 columns = stds_df.columns)
    
    # Calculate the new standard deviations 
    # Note: if x = a/b, then SD_x/x = sqrt((SD_a/a)^2 + (SD_b/b)^2)
    # Loop through each sample 
    for i in range(len(stds_df.columns)): 
        
        # Loop through each library member 
        for j in range(len(stds_df.index)): 
                   
            # Obtain the sample SD and mean for that library member
            SD_a = stds_df.iloc[j,i]
            a = avgs_df.iloc[j,i]
            
            # Obtain the No Serum SD and mean for that library member
            SD_b = no_serum_df['No Serum Pooled SD'][j]
            b = no_serum_df['No Serum Pooled Avg'][j]
                   
            # Obtain the new average (the ratio of the sample average to the No Serum average)
            x = no_serum_norm_avg_df.iloc[j,i]
            
            # Check that values are being assigned  
            # print('SD_a: ' + str(SD_a) + ' | a: ' + str(a) + 
            #       ' |SD_b: ' + str(SD_b) + ' | b: ' + str(b) +  
            #       ' | x: ' + str(x))
                   
            # Print if any values are zero 
            # if a == 0: 
            #     print('a is zero for ' + stds_df.columns[i] + ' ' + stds_df.index[j])
            # if b == 0: 
            #     print('b is zero for ' + no_serum_df.index[j])
                
            # Calculate the new standard deviation 
            SD_x = round(x * math.sqrt((SD_a/a)**2 + (SD_b/b)**2),2)
            
            # Check that calculations are working 
            # print('SD_x: ' + str(SD_x))
                   
            # Add the new standard deviation to the new dataframe 
            no_serum_stds.iloc[j,i] = SD_x
        
    return no_serum_stds 

In [ ]:
# Calculate the standard deviations for the double-normalized barcode fractions
stds_no_serum_norm = calc_no_serum_error_prop(norm_avgs, norm_stds, 
                                              no_serum_avg_std, 
                                              avgs_no_serum_norm)

# Note: certain library members' ladder-normalized barcode fractions may be 0 for the No Serum samples; 
#       this will prevent double normalization. The library members can be omitted from additional analyses.

In [ ]:
# Check which samples cannot be double-normalized 
print(norm_avgs[norm_avgs.eq(0).any(axis = 1)])
print(norm_stds[norm_stds.eq(0).any(axis = 1)])

# Note: these commands will also show which library members 
#       have ladder-normalized barcode fractions of 0 for the No Serum samples. 
#       The non-spike SARS-CoV-2 library members often have ladder-normalized 
#       barcode fractions of 0 for the No Serum samples. 

In [ ]:
# Save the double-normalized average and standard deviation dataframes
avgs_no_serum_norm.to_csv('DATE SEQUENCING_RUN_NAME Double-Normalized Barcode Fractions - Averages.csv')
stds_no_serum_norm.to_csv('DATE SEQUENCING_RUN_NAME Double-Normalized Barcode Fractions - Standard Deviations.csv')

# Compute the Log2FC for the ladder-normalized and double-normalized barcode fractions

In [ ]:
# Define a function to compute the Log2FC values from normalized values for every sample 

# FUNCTION PARAMETERS
# avgs_df: dataframe with averages of normalized barcode fractions 
# stds_df: dataframe with standard deviations of normalized barcode fractions

# Note: these two dataframes must match in their dimensions, indices, and columns. 

def calc_log2fc(avgs_df, stds_df): 
    
    # Convert the averages 
    new_avgs_df = round(np.log2(avgs_df),2)
    
    # Create an empty dataframe to store the new standard deviations 
    new_stds_df = pd.DataFrame(index = stds_df.index,
                               columns = stds_df.columns)
    
    # Loop through the dataframe and use log error propagation to calculate new standard deviations 
    # If x = log_2(a) --> SD_x = SD_a / (a * ln(2))
    for n in range(len(stds_df.index)): 
        for m in range(len(stds_df.columns)): 
            
            # Determine if there will be a divide-by-zero error 
            if avgs_df.iloc[n,m] == 0: 
                print('The average for ' + str(avgs_df.columns[m]) + ' ' + str(avgs_df.index[n]) + 
                      ' is 0.')
            
            # Calculate the SD 
            new_stds_df.iloc[n,m] = round(stds_df.iloc[n,m] / (avgs_df.iloc[n,m] * np.log(2)),2)
            
    return new_avgs_df, new_stds_df

In [ ]:
# Log2-transform the averages and standard deviations of the normalized barcode fractions 
# using log error propagation

# For the ladder-normalized barcode fractions 
log2_norm_avgs, log2_norm_stds = calc_log2fc(norm_avgs, norm_stds)

# For the double-normalized barcode fractions 
log2_no_serum_norm_avgs, log2_no_serum_norm_stds = calc_log2fc(avgs_no_serum_norm, stds_no_serum_norm)

# Note: The non-spike SARS-CoV-2 library members often cannot be log2-transformed
#       as they often have normalized barcode fractions of 0. 

In [ ]:
# Save the log2-transformed average and standard deviation dataframes
log2_norm_avgs.to_csv('DATE SEQUENCING_RUN_NAME Log2 Double-Normalized Barcode Fractions - Avg.csv')
log2_norm_stds.to_csv('DATE SEQUENCING_RUN_NAME Log2 Double-Normalized Barcode Fractions - SD.csv')

log2_no_serum_norm_avgs.to_csv('DATE SEQUENCING_RUN_NAME Log2 Double-Normalized Barcode Fractions - Avg.csv')
log2_no_serum_norm_stds.to_csv('DATE SEQUENCING_RUN_NAME Log2 Double-Normalized Barcode Fractions - SD.csv')